In [ ]:
weighted_network = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 3},
    'D': {},  # No connections
    'E': {'B': 5, 'F': 1},
    'F': {'C': 3, 'E': 1}
}

In [ ]:
# BFS
def bfs(source, sink):
    """Check if a path exists"""
    queue = [source]
    visited = set()

    while queue:
        # pop from top of queue
        current_node = queue.pop(0)
        
        if current_node == sink:
            return True
        
        if current_node not in visited:
            visited.add(current_node)
            for neighbor in weighted_network[current_node]:
                if neighbor not in visited:
                    # append to end of queue
                    queue.append(neighbor)

    return False


bfs('A', 'F')
# bfs('D', 'A')

In [ ]:
# DFS
def dfs(source, sink):
    stack = [source]
    visited = set()
    while stack:
        # get from end
        current_node = stack.pop()
        if current_node == sink:
            return True
        
        visited.add(current_node)
        for neighbor in weighted_network[current_node]:
            if neighbor not in visited and neighbor not in stack:
                stack.append(neighbor)
    return False

dfs('A', 'F')
# dfs('D', 'A')

In [ ]:
import heapq

# Dijkstra's Algorithm
def dike(source, sink):
    # init
    priority_queue = [(0, source, None)] # (total_distance, node, from_start)
    visited_nodes = set()
    distances = {node: float('inf') for node in weighted_network}
    distances[source] = 0
    
    # start with nearby neighbours
    ## update to distance from start
    ## after trying all neighbours then pick the shortest new one
    ### when we reached the goal and we know the shortest path stop
    shortest_path = float('inf')
    while priority_queue:
        # get next node
        current_dist, node, prev = heapq.heappop(priority_queue)
        
        if node in visited_nodes:
            continue
        
        visited_nodes.add(node) # mark as visited

        # if made it to the sink, we found the shortest path
        if node == sink:
            shortest_path = current_dist
            break
            
        # inspect neighbours
        for neigh, weight in weighted_network[node].items():
            neigh_dist = current_dist + weight
            
            if neigh not in visited_nodes and neigh_dist < distances[neigh]:
                distances[neigh] = neigh_dist
                heapq.heappush(priority_queue, (neigh_dist, neigh, node))
                
    return shortest_path

print(dike('A', 'B'))  # Expected output: 1


In [ ]:
# A*
## basically same as above but you now change it to include to distance to goal

import heapq

# Define the heuristic function
def heuristic(node, sink):
    heuristic_values = {
        'A': 2,
        'B': 1,
        'C': 2,
        'D': 3,
        'E': 1,
        'F': 0  # Assuming F is close to the sink in this example
    }
    return heuristic_values.get(node, 0)

# A* Algorithm
def a_star(source, sink):
    # Init
    priority_queue = [(0 + heuristic(source, sink), 0, source, None)] # (total_cost, total_distance, node, from_start)
    visited_nodes = set()
    distances = {node: float('inf') for node in weighted_network}
    distances[source] = 0
    
    shortest_path = float('inf')
    while priority_queue:
        # Get next node
        estimated_total_cost, current_dist, node, prev = heapq.heappop(priority_queue)
        
        if node in visited_nodes:
            continue
        
        visited_nodes.add(node) # Mark as visited

        # If made it to the sink, we found the shortest path
        if node == sink:
            shortest_path = current_dist
            break
            
        # Inspect neighbours
        for neigh, weight in weighted_network[node].items():
            neigh_dist = current_dist + weight
            
            if neigh not in visited_nodes and neigh_dist < distances[neigh]:
                distances[neigh] = neigh_dist
                total_cost = neigh_dist + heuristic(neigh, sink)
                heapq.heappush(priority_queue, (total_cost, neigh_dist, neigh, node))
                
    return shortest_path

print(a_star('A', 'B'))  # The output should reflect the use of the heuristic

In [ ]:
directed_network = {
    'S': {'V': 3, 'W': 2},
    'V': {'W': 5, 'T': 2},
    'W': {'T': 3},
}

In [ ]:
def dfs_recursive(graph, start, end, visited=None, pathway=None, existing_pathways=None):
    if not pathway:
        pathway = [start]
    if not visited:
        visited = {start}
    if existing_pathways is None:
        existing_pathways = []

    if pathway[-1] == end and pathway not in existing_pathways:
        existing_pathways.append(list(pathway))
        yield list(pathway)

    for neighbour in graph.get(start, []):
        if neighbour not in visited:
            visited.add(neighbour)
            pathway.append(neighbour)
            yield from dfs_recursive(graph, neighbour, end, visited, pathway, existing_pathways)
            visited.remove(neighbour)
            pathway.pop()

In [ ]:
def greedy_fordfulk(graph, source, sink):
    max_flow = 0
    capacity_network = graph.copy()

    for new_path in dfs_recursive(graph, source, sink):        
        # Find bottleneck
        node_pairs = []
        min_capacity = float('Inf') # can be 0 for blocked paths
        for i in range(len(new_path) - 1):
            node_from, node_to = new_path[i], new_path[i+1]
            node_pairs.append((node_from, node_to))
            min_capacity = min(min_capacity, capacity_network[node_from][node_to])

        # Apply it
        for nto, nfrom in node_pairs:
            capacity_network[nto][nfrom] -= min_capacity
        
        max_flow += min_capacity

    return max_flow

In [ ]:
greedy_fordfulk(directed_network, 'S', 'T')

In [ ]:
directed_network = {
    'S': {'V': 3, 'W': 2},
    'V': {'W': 5, 'T': 2},
    'W': {'T': 3},
}

In [ ]:
def fordfulk(source, sink):
    max_flow = 0

    residual_graph = {}
    for u in directed_network:
        if u not in residual_graph:
            residual_graph[u] = {}
        for v in directed_network[u]:
            if v not in residual_graph:
                residual_graph[v] = {}
            if v not in residual_graph[u]:
                residual_graph[u][v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = {}
            residual_graph[v][u] = 0
            residual_graph[u][v] = directed_network[u][v]

    for new_path in dfs_recursive(residual_graph, source, sink):

        # Find bottleneck capacity
        min_capacity = float('inf')
        for i in range(len(new_path) - 1):
            u, v = new_path[i], new_path[i+1]
            min_capacity = min(min_capacity, residual_graph[u][v])

        if not min_capacity:
            continue    
                
        # Update residual graph
        for i in range(len(new_path) - 1):
            u, v = new_path[i], new_path[i+1]
            residual_graph[u][v] -= min_capacity
            residual_graph[v][u] += min_capacity
        
        max_flow += min_capacity

    return max_flow

In [ ]:
fordfulk('S', 'T')